In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
import numpy as np
import warnings
import sys
import math

warnings.filterwarnings('ignore')

sys.path.append("..")

In [3]:
data_cl = pd.read_csv(f'../data/csgo_taskv2.csv')
data_rg = pd.read_csv(f'../data/trip_duration_taskv2.csv')

Data_CL

In [4]:
data_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122410 entries, 0 to 122409
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        122410 non-null  int64  
 1   time_left         122410 non-null  float64
 2   ct_score          122410 non-null  int64  
 3   t_score           122410 non-null  int64  
 4   map               122410 non-null  object 
 5   bomb_planted      122410 non-null  bool   
 6   ct_health         122410 non-null  int64  
 7   t_health          122410 non-null  int64  
 8   ct_armor          122410 non-null  int64  
 9   t_armor           122410 non-null  int64  
 10  ct_money          122410 non-null  int64  
 11  t_money           122410 non-null  int64  
 12  ct_helmets        122410 non-null  int64  
 13  t_helmets         122410 non-null  int64  
 14  ct_defuse_kits    122410 non-null  int64  
 15  ct_players_alive  122410 non-null  int64  
 16  t_players_alive   12

In [5]:
data_cl.loc[data_cl['bomb_planted'] == True, 'bomb_planted'] = 1
data_cl.loc[data_cl['bomb_planted'] == False, 'bomb_planted'] = 0

In [6]:
data_cl.head(10)

,Unnamed: 0,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive
0,0,175.00,0,0,de_dust2,0,500,500,0,0,4000,4000,0,0,0,5,5
1,1,156.03,0,0,de_dust2,0,500,500,400,300,600,650,0,0,1,5,5
2,2,96.03,0,0,de_dust2,0,391,400,294,200,750,500,0,0,1,4,4
3,3,76.03,0,0,de_dust2,0,391,400,294,200,750,500,0,0,1,4,4
4,4,174.97,1,0,de_dust2,0,500,500,192,0,18350,10750,0,0,1,5,5
5,5,114.97,1,0,de_dust2,0,500,500,500,0,1750,10550,5,0,1,5,5
6,6,94.97,1,0,de_dust2,0,295,111,296,0,1700,5000,3,0,0,3,2
7,7,74.97,1,0,de_dust2,0,295,111,296,0,1700,5000,3,0,0,3,2
8,8,54.97,1,0,de_dust2,0,249,89,269,0,2300,2950,3,0,0,3,1
9,9,174.97,2,0,de_dust2,0,500,500,196,0,19800,23450,2,0,0,5,5


In [7]:
data_cl = data_cl.drop(['map'], axis=1)

In [8]:
data_cl.isna().sum()

Unnamed: 0          0
time_left           0
ct_score            0
t_score             0
bomb_planted        0
ct_health           0
t_health            0
ct_armor            0
t_armor             0
ct_money            0
t_money             0
ct_helmets          0
t_helmets           0
ct_defuse_kits      0
ct_players_alive    0
t_players_alive     0
dtype: int64

In [9]:
data_major_true = data_cl[data_cl['bomb_planted'] == 1]
data_major_false = data_cl[data_cl['bomb_planted'] == 0]

data_major_false_undersampled = data_major_false.sample(len(data_major_true))
data_cl = pd.concat([data_major_false_undersampled, data_major_true], axis = 0)

y_cl = data_cl['bomb_planted']
for unique in y_cl.unique():
    print(unique, y_cl.loc[y_cl == unique].count())

0 13684
1 13684


In [10]:
x_cl = data_cl.drop(['bomb_planted'], axis=1)

In [11]:
x_train_cl, x_test_cl, y_train_cl, y_test_cl = train_test_split(x_cl, y_cl, test_size=0.3)

Data_RG

In [12]:
data_rg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         729322 non-null  int64  
 1   id                 729322 non-null  object 
 2   vendor_id          729322 non-null  int64  
 3   passenger_count    729322 non-null  int64  
 4   pickup_longitude   729322 non-null  float64
 5   pickup_latitude    727475 non-null  float64
 6   dropoff_longitude  729322 non-null  float64
 7   dropoff_latitude   729322 non-null  float64
 8   trip_duration      729322 non-null  int64  
 9   pickup_datetime    729322 non-null  object 
 10  dropoff_datetime   729322 non-null  object 
dtypes: float64(4), int64(4), object(3)
memory usage: 61.2+ MB


In [13]:
data_rg.isna().sum()

Unnamed: 0              0
id                      0
vendor_id               0
passenger_count         0
pickup_longitude        0
pickup_latitude      1847
dropoff_longitude       0
dropoff_latitude        0
trip_duration           0
pickup_datetime         0
dropoff_datetime        0
dtype: int64

In [14]:
mean_r = -0.0008561123804995453
for i in range(len(data_rg['pickup_latitude'])):
    if math.isnan(data_rg['pickup_latitude'].loc[data_rg.index[i]]):
        data_rg['pickup_latitude'].loc[data_rg.index[i]] = data_rg['dropoff_latitude'].loc[data_rg.index[i]] + mean_r
data_rg.isna().sum()

Unnamed: 0           0
id                   0
vendor_id            0
passenger_count      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
trip_duration        0
pickup_datetime      0
dropoff_datetime     0
dtype: int64

In [15]:
data_rg = data_rg.drop(['id'], axis=1)
data_rg = data_rg.drop(['pickup_datetime'], axis=1)
data_rg = data_rg.drop(['dropoff_datetime'], axis=1)

In [16]:
y_rg = data_rg['trip_duration']
x_rg = data_rg.drop(['trip_duration'], axis=1)

In [17]:
x_rg = x_rg[:10000]
y_rg = y_rg[:10000]

In [18]:
x_train_rg, x_test_rg, y_train_rg, y_test_rg = train_test_split(x_rg, y_rg, test_size=0.3)

In [19]:
# Regression
model_regression = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(7,)),
        tf.keras.layers.Dense(32, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)
model_regression.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                512       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3137 (12.25 KB)
Trainable params: 3137 (12

In [20]:
# compile
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

In [21]:
model_regression.fit(x_train_rg, y_train_rg, epochs=100)

Epoch 1/100
219/219 [==============================] - 1s 2ms/step - loss: 8288633.5000
Epoch 2/100
219/219 [==============================] - 0s 2ms/step - loss: 8226929.0000
Epoch 3/100
219/219 [==============================] - 0s 2ms/step - loss: 8193835.0000
Epoch 4/100
219/219 [==============================] - 0s 2ms/step - loss: 8168189.0000
Epoch 5/100
219/219 [==============================] - 0s 2ms/step - loss: 8179915.0000
Epoch 6/100
219/219 [==============================] - 0s 2ms/step - loss: 8142957.0000
Epoch 7/100
219/219 [==============================] - 0s 2ms/step - loss: 8158745.0000
Epoch 8/100
219/219 [==============================] - 0s 2ms/step - loss: 8167668.0000
Epoch 9/100
219/219 [==============================] - 0s 2ms/step - loss: 8149027.5000
Epoch 10/100
219/219 [==============================] - 0s 2ms/step - loss: 8165517.5000
Epoch 11/100
219/219 [==============================] - 0s 2ms/step - loss: 8162797.5000
Epoch 12/100
219/219 [========

In [22]:
print(mean_absolute_error(y_test_rg, model_regression.predict(x_test_rg)))
print(mean_squared_error(y_test_rg, model_regression.predict(x_test_rg)))

94/94 [==============================] - 0s 1ms/step
616.5218331502278
94/94 [==============================] - 0s 1ms/step
8087912.012202844


In [23]:
x_train_cl = x_train_cl.astype(np.float64)
y_train_cl = y_train_cl.astype(np.float64)

In [24]:

model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(15,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy")
model_classification_1.fit(x_train_cl, y_train_cl, epochs=100, verbose=None)

In [25]:
y_pred = np.around(model_classification_1.predict(x_test_cl, verbose=None))
print(classification_report(y_test_cl.astype(np.float64), y_pred))

              precision    recall  f1-score   support

         0.0       1.00      0.84      0.91      4111
         1.0       0.86      1.00      0.93      4100

    accuracy                           0.92      8211
   macro avg       0.93      0.92      0.92      8211
weighted avg       0.93      0.92      0.92      8211



In [26]:
# 2
model_classification_2 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_dim = x_cl.shape[1]),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)

model_classification_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="sparse_categorical_crossentropy")
model_classification_2.fit(x_train_cl, y_train_cl, epochs=100, verbose=None)

In [28]:
y_pred = [np.argmax(pred) for pred in model_classification_2.predict(x_test_cl, verbose=None)]
print(classification_report(y_test_cl.astype(np.float64), y_pred))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      4111
         1.0       0.50      1.00      0.67      4100

    accuracy                           0.50      8211
   macro avg       0.25      0.50      0.33      8211
weighted avg       0.25      0.50      0.33      8211

